In [1]:
from deeppavlov import build_model, configs

In [3]:
from pathlib import Path
CONFIG_PATH = Path('./movie-review-sentiment-analysis_ag_news_wiki.json')  # could also be configuration dictionary or string path or `pathlib.Path` instance

In [44]:
! cat {CONFIG_PATH}

{
    "dataset_reader": {
      "class_name": "basic_classification_reader",
      "x": "Phrase",
      "y": "Sentiment",
      "format": "csv",
      "sep" : "\t",
      "data_path": "{DOWNLOADS_PATH}/moview_review_sentiment_analysis_ag_news_wiki",
      "train": "train.tsv",
      "test": "test.tsv",
      "header": 0,
      "names": [
        "Phrase",
        "Sentiment"
      ]
    },
    "dataset_iterator": {
      "class_name": "basic_classification_iterator",
      "seed": 42,
      "split_seed": 23,
      "field_to_split": "train",
      "split_fields": [
        "train",
        "valid"
      ],
      "split_proportions": [
        0.9,
        0.1
      ]
    },
     "chainer": {
    "in": [
      "x"
    ],
    "in_y": [
      "y"
    ],
    "pipe": [
      {
        "id": "classes_vocab",
        "class_name": "simple_vocab",
        "fit_on": [
          "y"
        ],
        "save_path": "{MODEL_PATH}/classes.dict",
        "load_path": "{MODEL_PATH}/classes.dict",
    

In [4]:
model = build_model(CONFIG_PATH, download=False)  # in case of necessity to download some data

2020-03-29 13:33:07.872 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /home/kuptservol/.deeppavlov/models/classifiers/moview_review_sentiment_analysis_ag_news_wiki/classes.dict]
/home/kuptservol/.pyenv/versions/3.6.3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/kuptservol/.pyenv/versions/3.6.3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/kuptservol/.pyenv/versions/3.6.3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/kuptservol/.pyenv/versions/3.6.3/lib/python3.6/import

2020-03-29 13:33:21.712 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 273: [initializing `KerasClassificationModel` from saved]


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2020-03-29 13:33:22.154 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 283: [loading weights from model.h5]



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2020-03-29 13:33:22.461 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 134: Model was successfully initialized!
Model summary:
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 300)    0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 256)    230656      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, None, 256)    384256      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)      

In [8]:
model(['I hate it!'])

['2']

In [19]:
data_path = Path('~/.kaggle/data/sentiment-analysis-on-movie-reviews/')

In [20]:
! ls {data_path}

sampleSubmission.csv  test.tsv	train.tsv


In [25]:
import pandas as pd
test_csv_pd = pd.read_csv(data_path/'test.tsv', delimiter='\t'); test_csv_pd.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [26]:
answers = test_csv_pd['Phrase'].apply(lambda x: model([x]))

In [30]:
test_csv_pd['Phrase'][0]

'An intermittently pleasing but mostly routine effort .'

In [31]:
submission = pd.DataFrame()

In [32]:
submission['PhraseId'] = test_csv_pd['PhraseId'].astype(int)

In [33]:
submission.head()

,PhraseId
0,156061
1,156062
2,156063
3,156064
4,156065


In [38]:
submission['Sentiment'] = answers.apply(lambda x: x[0])

In [39]:
submission.head()

,PhraseId,Sentiment
0,156061,3
1,156062,3
2,156063,2
3,156064,3
4,156065,3


In [40]:
submission.shape


(66292, 2)

In [42]:
submission.to_csv(data_path/'submission.csv', index=False)

In [43]:
! kaggle competitions submit sentiment-analysis-on-movie-reviews -f {data_path/'submission.csv'} -m "My submission"

100%|█████████████████████████████████████████| 583k/583k [00:04<00:00, 142kB/s]
Successfully submitted to Sentiment Analysis on Movie Reviews